In [1]:
import numpy as np
import monkeypuck as mp
import pandas as pd
from scipy.stats import poisson
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import requests_cache
  

In [2]:
requests_cache.install_cache()
agg_columns = [
            "gameId",
            "opposingTeam",
            "gameDate",
            "season",
            "xGoalsPercentage",
            "team",
            "situation",
            "xGoalsPercentage",
            "xOnGoalFor",
            "xGoalsFor",
            "goalsFor",
            "shotsOnGoalFor",
            "xGoalsAgainst",
            "shotsOnGoalAgainst",
            "goalsAgainst",
            "xOnGoalAgainst",
        ]

In [42]:
df = mp.MonkeyPuck().xg_df
df = df[df['situation'] == 'all']
df.drop(df[df['iceTime'] > 3600].index)
agg_columns = [
            "xGoalsPercentage",
            "xGoalsPercentage",
            "xOnGoalFor",
            "xGoalsFor",
            "goalsFor",
            "shotsOnGoalFor",
            "xGoalsAgainst",
            "shotsOnGoalAgainst",
            "goalsAgainst",
            "xOnGoalAgainst",
        ]
df = pd.get_dummies(df,columns=['home_or_away'])
df = df.rename(columns={'home_or_away_HOME': 'home'}).drop(columns=['home_or_away_AWAY'])
df['teamId'] = np.int64(pd.Categorical(df.team).codes)
df['opposingTeamId'] = np.int64(pd.Categorical(df.opposingTeam).codes)
table = df.sort_values(['teamId', 'gameDate'], ascending=(True,True))
table1 = table.assign(total_games=table.groupby('teamId', sort=False)['home'].transform(lambda x: x.expanding().count() - 1))
table1['previous_expected_goals_for'] = table1['xGoalsFor'].shift(1)
print(table1.head())
table1['previous_expected_goals_for2'] = table1['xGoalsFor'].shift(2)
table1['previous_expected_goals_for3'] = table1['xGoalsFor'].shift(3)
table1['previous_expected_goals_for4'] = table1['xGoalsFor'].shift(4)
table1['previous_expected_goals_for5'] = table1['xGoalsFor'].shift(5)
table1['previous_team_id'] = table1['teamId'].shift(1)
table1.loc[table1['previous_team_id'] != table1['teamId'], 'previous_expected_goals_for'] = 0
table1 = table1.drop(table1[table1['total_games'] == 0].index)
table1 = table1.assign(average_goals=table1.groupby('teamId', sort=False)['previous_expected_goals_for'].transform(lambda x: x.rolling(5,min_periods=2).mean()),average_goals2=table1.groupby('teamId', sort=False)['previous_expected_goals_for'].transform(lambda x: x.rolling(10,min_periods=5).mean()))

# print((table1[table1['team'] == 'NYR'])[['gameDate','home','total_games','xGoalsFor','goalsFor','average_goals','previous_expected_goals_for']].head(10))
print(df.columns)

# #sort by kicker id and play id chronologically to set it up as a time series
# field_goals_for_kicker_analysis = field_goals_for_kicker_analysis.sort_values(['kicker_id','play_id'], ascending=(True,True))

# #shift field_goals_over_expectation and distance down a row to calculate out of sample kicker analytics (we do not want to include the current kick in our analytics since that data should be unseen)
# field_goals_for_kicker_analysis['previous_field_goals_over_expectation'] = field_goals_for_kicker_analysis['field_goals_over_expectation'].shift(1)
# field_goals_for_kicker_analysis['previous_distance'] = field_goals_for_kicker_analysis['distance'].shift(1)

# #add previous kicker id so we can filter out the first new kick for a kicker
# field_goals_for_kicker_analysis['previous_kicker_id'] = field_goals_for_kicker_analysis['kicker_id'].shift(1)
# field_goals_for_kicker_analysis.loc[field_goals_for_kicker_analysis['previous_kicker_id'] != field_goals_for_kicker_analysis['kicker_id'], 'previous_field_goals_over_expectation'] = 0
# field_goals_for_kicker_analysis.loc[field_goals_for_kicker_analysis['previous_kicker_id'] != field_goals_for_kicker_analysis['kicker_id'], 'previous_distance'] = 0

# #add column calculating total field goals up to that point in each kicker's data set, excluding current field goal attempt
# field_goals_for_kicker_analysis = field_goals_for_kicker_analysis.assign(total_field_goals=field_goals_for_kicker_analysis.groupby('kicker_id', sort=False)['made'].transform(lambda x: x.expanding().count() - 1))

# #drop first kick for each kicker since we have no data for it
# field_goals_for_kicker_analysis = field_goals_for_kicker_analysis.drop(field_goals_for_kicker_analysis[field_goals_for_kicker_analysis['total_field_goals'] == 0].index)

# #add column calculating average field goals over expectation up to that point in each kicker's data set, excluding current field goal attempt
# field_goals_for_kicker_analysis = field_goals_for_kicker_analysis.assign(kicker_accuracy=field_goals_for_kicker_analysis.groupby('kicker_id', sort=False)['previous_field_goals_over_expectation'].transform(lambda x: x.expanding().mean()))

# #add column calculating average of top 5 longest field goals up to that point in each kicker's data set, excluding current field goal attempt
# field_goals_for_kicker_analysis = field_goals_for_kicker_analysis.assign(kicker_leg_strength=field_goals_for_kicker_analysis.groupby('kicker_id', sort=False)['previous_distance'].transform(lambda x: x.expanding().apply(lambda n: n.nlargest(5).mean())))

# #drop shifted columns
# field_goals_for_kicker_analysis.drop(columns=['previous_field_goals_over_expectation', 'previous_distance', 'previous_kicker_id'], inplace=True)

           gameId opposingTeam  gameDate  season  xGoalsPercentage team  \
61881  2022020009          SEA  20221012    2022            0.3188  ANA   
61886  2022020032          NYI  20221015    2022            0.3821  ANA   
61891  2022020039          NYR  20221017    2022            0.3032  ANA   
61896  2022020048          NJD  20221018    2022            0.2798  ANA   
61901  2022020059          BOS  20221020    2022            0.3541  ANA   

      situation  xGoalsPercentage  xOnGoalFor  xGoalsFor  ...  xGoalsAgainst  \
61881       all            0.3188      22.921      2.356  ...          5.035   
61886       all            0.3821      26.321      2.389  ...          3.863   
61891       all            0.3032      18.361      1.628  ...          3.741   
61896       all            0.2798      20.078      1.506  ...          3.876   
61901       all            0.3541      28.544      1.543  ...          2.815   

       shotsOnGoalAgainst  goalsAgainst  xOnGoalAgainst  iceTime  ho

,gameId,opposingTeam,gameDate,season,xGoalsPercentage,team,situation,xGoalsPercentage,xOnGoalFor,xGoalsFor,...,opposingTeamId,total_games,previous_expected_goals_for,previous_expected_goals_for2,previous_expected_goals_for3,previous_expected_goals_for4,previous_expected_goals_for5,previous_team_id,average_goals,average_goals2
5386,2022020017,MIN,20221013,2022,0.4152,NYR,all,0.4152,30.783,3.165,...,14,1.0,5.589,2.549,2.495,2.101,3.344,19.0,NaN,NaN
5391,2022020023,WPG,20221014,2022,0.4000,NYR,all,0.4000,40.542,2.814,...,30,2.0,3.165,5.589,2.549,2.495,2.101,19.0,4.37700,NaN
5396,2022020039,ANA,20221017,2022,0.6968,NYR,all,0.6968,41.567,3.741,...,0,3.0,2.814,3.165,5.589,2.549,2.495,19.0,3.85600,NaN
5401,2022020064,SJS,20221020,2022,0.4877,NYR,all,0.4877,30.029,2.970,...,24,4.0,3.741,2.814,3.165,5.589,2.549,19.0,3.82725,NaN
5406,2022020089,CBJ,20221023,2022,0.5683,NYR,all,0.5683,34.020,2.571,...,5,5.0,2.970,3.741,2.814,3.165,5.589,19.0,3.65580,3.655800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165131,2022020130,PHI,20221029,2022,0.5138,CAR,all,0.5138,41.818,3.316,...,21,7.0,3.682,3.594,3.037,3.675,3.404,4.0,3.47840,3.371857
165136,2022020145,WSH,20221031,2022,0.6819,CAR,all,0.6819,38.180,4.148,...,31,8.0,3.316,3.682,3.594,3.037,3.675,4.0,3.46080,3.364875
165141,2022020162,TBL,20221103,2022,0.5760,CAR,all,0.5760,51.223,4.471,...,26,9.0,4.148,3.316,3.682,3.594,3.037,4.0,3.55540,3.451889
165146,2022020174,BUF,20221104,2022,0.5370,CAR,all,0.5370,34.766,3.826,...,3,10.0,4.471,4.148,3.316,3.682,3.594,4.0,3.84220,3.553800


In [43]:
poisson_training_results = sm.GLM(table1['goalsFor'], table1[['average_goals','average_goals2','previous_expected_goals_for','previous_expected_goals_for2','previous_expected_goals_for3','previous_expected_goals_for4','previous_expected_goals_for5']], family=sm.families.Poisson(), missing='drop').fit()

In [44]:
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               goalsFor   No. Observations:                  252
Model:                            GLM   Df Residuals:                      246
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -492.93
Date:                Mon, 21 Nov 2022   Deviance:                       278.03
Time:                        23:36:25   Pearson chi2:                     260.
No. Iterations:                    39   Pseudo R-squ. (CS):           -0.06905
Covariance Type:            nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
average_goals   

In [45]:
df.sort_values(['teamId', 'gameDate'], ascending=(True,True)).groupby('teamId',sort=False).agg(lambda x: list(x))

,gameId,opposingTeam,gameDate,season,xGoalsPercentage,team,situation,xGoalsPercentage,xOnGoalFor,xGoalsFor,goalsFor,shotsOnGoalFor,xGoalsAgainst,shotsOnGoalAgainst,goalsAgainst,xOnGoalAgainst,iceTime,home,opposingTeamId
teamId,,,,,,,,,,,,,,,,,,,
0,"[2022020009, 2022020032, 2022020039, 202202004...","[SEA, NYI, NYR, NJD, BOS, DET, TBL, VGK, TOR, ...","[20221012, 20221015, 20221017, 20221018, 20221...","[2022, 2022, 2022, 2022, 2022, 2022, 2022, 202...","[0.3188, 0.3821, 0.3032, 0.2798, 0.3541, 0.378...","[ANA, ANA, ANA, ANA, ANA, ANA, ANA, ANA, ANA, ...","[all, all, all, all, all, all, all, all, all, ...","[0.3188, 0.3821, 0.3032, 0.2798, 0.3541, 0.378...","[22.921, 26.321, 18.361, 20.078, 28.544, 33.83...","[2.356, 2.389, 1.628, 1.506, 1.543, 2.422, 1.6...","[5.0, 1.0, 4.0, 2.0, 1.0, 1.0, 2.0, 0.0, 4.0, ...","[27.0, 23.0, 22.0, 20.0, 31.0, 33.0, 21.0, 29....","[5.035, 3.863, 3.741, 3.876, 2.815, 3.978, 3.3...","[48.0, 38.0, 43.0, 41.0, 36.0, 41.0, 30.0, 22....","[4.0, 7.0, 6.0, 4.0, 1.0, 5.0, 4.0, 4.0, 3.0, ...","[44.26, 36.594, 41.567, 39.352, 33.018, 33.075...","[3655.0, 3600.0, 3600.0, 3600.0, 3900.0, 3600....","[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]","[23, 18, 19, 16, 2, 10, 26, 29, 27, 24, 28, 24..."
1,"[2022020011, 2022020027, 2022020040, 202202006...","[PIT, BOS, TOR, MTL, OTT, CBJ, WPG, NYR, FLA, ...","[20221013, 20221015, 20221017, 20221020, 20221...","[2022, 2022, 2022, 2022, 2022, 2022, 2022, 202...","[0.3089, 0.3015, 0.5019, 0.3226, 0.25, 0.4315,...","[ARI, ARI, ARI, ARI, ARI, ARI, ARI, ARI, ARI, ...","[all, all, all, all, all, all, all, all, all, ...","[0.3089, 0.3015, 0.5019, 0.3226, 0.25, 0.4315,...","[24.796, 24.468, 17.018, 26.885, 20.612, 21.49...","[2.68, 2.389, 3.304, 1.765, 1.223, 2.613, 1.86...","[2.0, 3.0, 4.0, 2.0, 2.0, 6.0, 2.0, 2.0, 3.0, ...","[28.0, 24.0, 19.0, 27.0, 22.0, 19.0, 21.0, 20....","[5.995, 5.534, 3.279, 3.707, 3.669, 3.442, 4.3...","[53.0, 43.0, 28.0, 30.0, 32.0, 33.0, 35.0, 39....","[6.0, 6.0, 2.0, 6.0, 6.0, 3.0, 3.0, 3.0, 1.0, ...","[47.905, 40.828, 31.296, 26.293, 29.537, 35.54...","[3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600....","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]","[22, 2, 27, 15, 20, 5, 30, 19, 12, 9, 31, 3]"
2,"[2022020005, 2022020027, 2022020041, 202202004...","[WSH, ARI, FLA, OTT, ANA, MIN, DAL, DET, CBJ, ...","[20221012, 20221015, 20221017, 20221018, 20221...","[2022, 2022, 2022, 2022, 2022, 2022, 2022, 202...","[0.5323, 0.6985, 0.549, 0.4741, 0.6459, 0.6453...","[BOS, BOS, BOS, BOS, BOS, BOS, BOS, BOS, BOS, ...","[all, all, all, all, all, all, all, all, all, ...","[0.5323, 0.6985, 0.549, 0.4741, 0.6459, 0.6453...","[28.285, 40.828, 33.054, 36.761, 33.018, 40.22...","[3.137, 5.534, 4.631, 3.554, 2.815, 4.335, 3.4...","[5.0, 6.0, 5.0, 5.0, 1.0, 4.0, 3.0, 5.0, 4.0, ...","[30.0, 43.0, 34.0, 34.0, 36.0, 43.0, 29.0, 33....","[2.756, 2.389, 3.804, 3.942, 1.543, 2.383, 2.5...","[35.0, 24.0, 41.0, 31.0, 31.0, 27.0, 31.0, 29....","[2.0, 3.0, 3.0, 7.0, 1.0, 3.0, 1.0, 1.0, 0.0, ...","[34.114, 24.468, 40.904, 34.103, 28.544, 25.96...","[3600.0, 3600.0, 3600.0, 3600.0, 3900.0, 3889....","[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]","[31, 1, 12, 20, 0, 14, 9, 10, 5, 22, 19, 27, 25]"
3,"[2022020013, 2022020025, 2022020054, 202202006...","[OTT, FLA, EDM, CGY, VAN, SEA, MTL, CHI, DET, ...","[20221013, 20221015, 20221018, 20221020, 20221...","[2022, 2022, 2022, 2022, 2022, 2022, 2022, 202...","[0.6325, 0.2828, 0.4613, 0.6072, 0.6027, 0.300...","[BUF, BUF, BUF, BUF, BUF, BUF, BUF, BUF, BUF, ...","[all, all, all, all, all, all, all, all, all, ...","[0.6325, 0.2828, 0.4613, 0.6072, 0.6027, 0.300...","[33.015, 24.036, 24.67, 38.449, 31.383, 17.359...","[3.753, 1.886, 4.458, 4.68, 4.0, 1.632, 3.687,...","[4.0, 3.0, 4.0, 6.0, 5.0, 1.0, 2.0, 4.0, 8.0, ...","[36.0, 27.0, 24.0, 32.0, 27.0, 16.0, 45.0, 45....","[2.181, 4.782, 5.205, 3.027, 2.637, 3.807, 2.8...","[36.0, 37.0, 48.0, 43.0, 30.0, 24.0, 34.0, 26....","[1.0, 4.0, 2.0, 3.0, 1.0, 5.0, 3.0, 3.0, 3.0, ...","[29.499, 39.569, 47.083, 37.868, 30.595,

1. Separate out test and train, will want clean data as we perform manipulations
2. Change mp data to be based on team, so then we can run based on game date. 
  - Decide columns. (start with simple xG of the game)
  - Rotate table so each column is called 'column_{date}'
  - get total and 5 game average
  - 
You have figured out how it works, now it’s time to finish your data APIs, so that it is easier to build without constantly messing with pandas. Where to begin?
1. Monkeypuck class is done. Read monkeypuck data into team profiles. 
2. Each team will have their games
3. Get games. And in a table print the defense versus offensive comparison. i.e:
Team1	xGFor
Team2	xGAgainst

